## Sorting and Grouping
The Search service offers the ability to organize your result set in different ways.  Whether you choose to sort based on a specific property, group properties or boost the result set, you have the option to override the default ordering returned. In addition, you can also use navigation to organize your buckets of data returned.

By default, the result set will provide some basic ranking, depending on the type of query or filter performed.  

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta4"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.20.1 Refinitiv.Data.Content, 1.0.0-beta4

Loading extensions from `C:\Users\U8007876\.nuget\packages\microsoft.data.analysis\0.20.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

2023-02-21 4:05:34 PM:DesktopSession. (State: DesktopSession is Pending)
2023-02-21 4:05:36 PM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
2023-02-21 4:05:36 PM:DesktopSession. (State: DesktopSession is Opened)


#### Example - Default ranking and how to boost results
To demonstrate default sorting, I'm going to search for documents that match a specific ticker. This will result in the documents ranked based on the relevancy and scores assigned to each document.

Search for Vodafone (VOD) against the ticker:

In [5]:
var response = Search.Definition(Search.View.SearchAll).Filter("TickerSymbol eq 'VOD'")
                                                       .Select("_, RCSExchangeCountryLeaf")
                                                       .GetData();
response.Data.Table

QUOTExEQUITY,"Vodafone Group PLC, Ordinary Share, London Stock Exchange",55836053361,1049261,VOD.L,United Kingdom
QUOTExEQUITY,"Vodacom Group Ltd, Ordinary Share, Johannesburg Stock Exchange",55849756323,59938825,VODJ.J,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Limited Cash Settled SSF Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21508525903,113268320,VODSc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Can Do VODX Vodacom Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21534036541,148452798,VODXFLc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group CFD Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21600821292,244233208,VODCFDc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Limited Cash Settled CFD Sabor Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21693141305,363600087,VODSCFDc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Ltd SSF Cash Settled Dividend Neutral Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21707726176,381845837,VODTc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodafone Group PLC Dividend Neutral Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21481060885,53826719,VODDc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodafone Group PLC IDX Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21481060918,53826752,VODGc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group SSF Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21481084690,60188651,VODQc1,South Africa


The above output ranked the common share 'VOD.L', listed within the UK, at the top.  This is largely due to the fact that 'VOD.L' is the most significant or liquid asset in the list.  However, if I decide to override this default ranking by specifying that I want to show the listed documents within the Unitied States at the top, I can do this 
by applying a *Boost* parameter.

In [6]:
response = Search.Definition(Search.View.SearchAll).Filter("TickerSymbol eq 'VOD'")
                                                   .Boost("RCSExchangeCountryLeaf eq 'United States'")
                                                   .Select("_, RCSExchangeCountryLeaf")
                                                   .GetData();
response.Data.Table

QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ Global Select Consolidated",55839401038,726233,VOD.O,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ Stock Exchange Global Select Market",55835364259,1092283,VOD.OQ,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe BZX Exchange - Nasdaq Global Select Market",55845638958,53705532,VOD.Z,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe EDGX Exchange - Nasdaq Global Select Market",21475045362,73238578,VOD.DG,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Boston SE when trading NASDAQ Global Select Market",55846134804,55957391,VOD.B,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NYSE Arca when trading NASDAQ Global Select Market",55837254988,754172,VOD.P,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe BYX Exchange - Nasdaq Global Select Market",21475122433,76383634,VOD.ZY,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe EDGA Exchange - Nasdaq Global Select Market",21475045135,73240457,VOD.DY,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ OMX PHLX Global Select Market",21475115114,76271998,VOD.PH,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NYSE National Exchange when trading Nasdaq Global Select Market",55838271132,735992,VOD.C,United States


#### Example - List the youngest CEO's.
The order_by parameters will sort, ascending (default) or descending, based on the birth year property.

Note: In the following example, not every document that identifies a CEO will have a reported birth year. Because we are sorting based on the year they were born, 
all CEO's where the year of birth is not recorded will be bumped to the bottom of the list.

In [7]:
var filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')";
var select = "FullName, YearOfBirth, DTCharacteristics, PrimaryOfficerDirectorRIC, PrimaryOfficerDirector";

response = Search.Definition(Search.View.People).Filter(filter)
                                                .OrderBy("YearOfBirth desc, LastName, FirstName")
                                                .Select(select)
                                                .GetData();
response.Data.Table

Hiroyasu Saito,1996,"Chief Executive Officer, Representative Director",,Gracia Inc
Connor Campbell,1994,"Chief Executive Officer, Co-Founder",,Osler Diagnostics Ltd
Ritesh Agarwal,1993,"Non-Executive Chairman of the Board, Founder",ORAV.NS,Oravel Stays Ltd
Tim Groot,1993,"Chief Executive Officer, Founder",,IntrosAt Ltd
Yiduitakuya Iwamoto,1993,"Chief Executive Officer, Co-Founder, Representative Director",,Polyuse Inc
Alberto Rizzoli,1993,Chief Executive Officer,,V7 Ltd
Tongda Wu,1993,"Chief Executive Officer, Co-Founder",,Paifang Technology Tianjin Co Ltd
Ross Bailey,1992,"Chief Executive Officer, Founder",,Appear Here Ltd
Ed Leon Klinger,1992,Chief Executive Officer,,Flock Ltd
Snigdha Mothukuri,1992,"Chief Executive Officer, Executive Director",JEEV.BO,Jeevan Scientific Technology Ltd


Instead of sorting by the year of birth, let's organize the output so the company is grouped.

**Note**: While we can see the company's grouped, we can also observe the many of the entries do not have a year of birth recorded.

In [8]:
filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')";
select = "FullName, YearOfBirth, DTCharacteristics, PrimaryOfficerDirectorRIC, PrimaryOfficerDirector";

response = Search.Definition(Search.View.People).Filter(filter)
                                                .GroupBy("PrimaryOfficerDirector")
                                                .Top(20)
                                                .Select(select)
                                                .GetData();
response.Data.Table

Satya Nadella,,"Chairman of the Board, Chief Executive Officer",MSFT.O,Microsoft Corp
Tim D. Cook,,"Chief Executive Officer, Director",AAPL.O,Apple Inc
Andrew R. Jassy,,"President, Chief Executive Officer, Director",AMZN.O,Amazon.com Inc
Sundar Pichai,1972,"Chief Executive Officer, Director",GOOGL.O,Alphabet Inc
R. Martin Chavez,,Director,GOOGL.O,Alphabet Inc
Mark Zuckerberg,,"Chairman of the Board, Chief Executive Officer, Founder",META.O,Meta Platforms Inc
Warren E. Buffett,1930,"Chairman of the Board, Chief Executive Officer",BRKa,Berkshire Hathaway Inc
Daniel Zhang,1972,"Chairman of the Board, Chief Executive Officer",BABA.K,Alibaba Group Holding Ltd
Huateng Ma,,"Executive Chairman of the Board, Chief Executive Officer",0700.HK,Tencent Holdings Ltd
Ryan McInerney,,"Chief Executive Officer, Director",V,Visa Inc
Joaquin Boix Duato,,"Chairman of the Board, Chief Executive Officer",JNJ,Johnson & Johnson


#### Example - Sorting using Navigators
By default, when you use a navigator against a property, it will sort all results based on the number of matches for each value within a bucket.  For example, if I were to list the top 10 exchanges within Canada, we can see the count value ranked, indicating the number instruments matched on that exchange.

In [9]:
response = Search.Definition(Search.View.EquityQuotes).Filter("RCSExchangeCountryLeaf eq 'Canada'")
                                                      .Top(0)
                                                      .Navigators("ExchangeName(buckets:10)")
                                                      .GetData();
Console.WriteLine(response.Data.Navigators["ExchangeName"]["Buckets"]);

[
  {
    "Label": "Montreal Options Exchange",
    "Count": 93904
  },
  {
    "Label": "Montreal Exchange",
    "Count": 32916
  },
  {
    "Label": "The Toronto Stock Exchange",
    "Count": 11709
  },
  {
    "Label": "Refinitiv",
    "Count": 10759
  },
  {
    "Label": "Nasdaq CXC",
    "Count": 10612
  },
  {
    "Label": "Canadian Composite Quote/Trade",
    "Count": 8671
  },
  {
    "Label": "Canadian Securities Exchange/PURE - CSE Listed & Other Canadian Listed Securities",
    "Count": 8652
  },
  {
    "Label": "Tradelogiq Omega ATS",
    "Count": 7915
  },
  {
    "Label": "Alpha - Canadian ATS",
    "Count": 7572
  },
  {
    "Label": "TSX Venture Exchange (former Canadian Ventures Exchange)",
    "Count": 7335
  }
]


Using the above example, I can instead choose to sort based on the average daily volume within the exchange.  The following search will result in the top 5 Canadian exchanges, ranked based on the 90 day average volume:

In [10]:
response = Search.Definition(Search.View.EquityQuotes).Filter("RCSExchangeCountryLeaf eq 'Canada'")
                                                      .Top(0)
                                                      .Navigators("ExchangeName(buckets:5, desc:sum_AvgVol90D)")
                                                      .GetData();

In [11]:
// Pretty display of the listing
var writer = new System.IO.StringWriter();
var headers = new List<IHtmlContent> {th("Exchange"), th("Average 90-day Volume")};
var rows = new List<IList<IHtmlContent>>();
        
foreach (var entry in response.Data.Navigators["ExchangeName"]["Buckets"])
{
    var vol = entry["sum_AvgVol90D"].ToObject<long>();
    var cells = new List<IHtmlContent> {td(entry["Label"].ToString()), td($"{vol:n0}")};
    rows.Add(cells);
}
var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
t

Canadian Composite Quote/Trade,"1,785,367,447"
The Toronto Stock Exchange,"1,071,986,564"
TSX Venture Exchange (former Canadian Ventures Exchange),"362,989,556"
Canadian Securities Exchange - CSE Listed,"102,126,774"
Canadian Securities Exchange/PURE - CSE Listed & Other Canadian Listed Securities,"43,979,250"
